In [179]:
#Import
import pandas as pd
import numpy as np
import torch
import re
from transformers import pipeline

In [180]:
#Sentiment Pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [181]:
#Import Tweets
df = pd.read_csv('COIN-Project/jupyter/tweets_from_accounts.csv')

#For Testing Purposes shorten to 3 Tweets
#df = df.head(100)

#For Oversight delete unused collums
#df = df[["id", "like_count", "text"]]
#df

In [182]:
def cleanText(x):
    #Delete Username
    x = re.sub("@[A-Za-z0-9_]+","", x)
    #Delete Hashtags
    x = re.sub("#[A-Za-z0-9_]+","", x)
    #Delete Links
    x = re.sub(r"http\S+","", x)
    x = re.sub(r"www.\S+","", x)
    #Remove all non-word characters (everything except numbers and letters)
    x = re.sub(r"[^\w\s]", ' ', x)
    #Remove Linebreaks and shit
    x = re.sub(r"[\n]", '', x)
    x = re.sub(r"[\r]", '', x)
    return x

In [183]:
df = df.assign(clean_text=df.apply(lambda x: cleanText(x['text']), axis=1))
df.head(3)

,id,author_id,created_at,geo,retweet_count,reply_count,like_count,qoute_count,text,entities,context_annotations,clean_text
0,1526540001995022337,1072187272815149057,2022-05-17 12:27:52+00:00,NaN,17,10,158,1,"Spread Love, not CO2! 🏳️‍🌈\r\n#IDAHOBIT2022 #Y...","{'hashtags': [{'start': 27, 'end': 40, 'tag': ...",[],Spread Love not CO2
1,1526536363755151360,1072187272815149057,2022-05-17 12:13:25+00:00,NaN,16,10,175,1,Quote of the day: Queer denken statt Querdenke...,"{'hashtags': [{'start': 49, 'end': 60, 'tag': ...",[],Quote of the day Queer denken statt Querdenken
2,1526534237972746242,1072187272815149057,2022-05-17 12:04:58+00:00,NaN,21,62,274,8,Was bedeutet queer?\r\n\r\nQueer ist eine posi...,"{'hashtags': [{'start': 246, 'end': 257, 'tag'...",[],Was bedeutet queer Queer ist eine positive Sel...


In [184]:
def senti_label(x):
    return sentiment_task(x)[0]["label"]

def senti_score(x):
    return sentiment_task(x)[0]["score"]

In [185]:
#New Collumn Sentiment
# [VERALTET] df = df.assign(sentiment = sentiment_task(df["text"].to_list()))

In [186]:
#New Sentiment Collumns
df = df.assign(senti_label=df.apply(lambda x: senti_label(x['clean_text']), axis=1))
df = df.assign(senti_score=df.apply(lambda x: senti_score(x['clean_text']), axis=1))
df.head(7)

,id,author_id,created_at,geo,retweet_count,reply_count,like_count,qoute_count,text,entities,context_annotations,clean_text,senti_label,senti_score
0,1526540001995022337,1072187272815149057,2022-05-17 12:27:52+00:00,NaN,17,10,158,1,"Spread Love, not CO2! 🏳️‍🌈\r\n#IDAHOBIT2022 #Y...","{'hashtags': [{'start': 27, 'end': 40, 'tag': ...",[],Spread Love not CO2,Neutral,0.615374
1,1526536363755151360,1072187272815149057,2022-05-17 12:13:25+00:00,NaN,16,10,175,1,Quote of the day: Queer denken statt Querdenke...,"{'hashtags': [{'start': 49, 'end': 60, 'tag': ...",[],Quote of the day Queer denken statt Querdenken,Neutral,0.571173
2,1526534237972746242,1072187272815149057,2022-05-17 12:04:58+00:00,NaN,21,62,274,8,Was bedeutet queer?\r\n\r\nQueer ist eine posi...,"{'hashtags': [{'start': 246, 'end': 257, 'tag'...",[],Was bedeutet queer Queer ist eine positive Sel...,Neutral,0.820247


In [187]:
df.to_csv(r'tweets_from_accounts_with_sentiment.csv', index = False)